# SHAPE FUNCTION

In [ ]:
# Created by K.Nakamura on 2015-11-21.
# e-mail: nakamura-keita-kn@ynu.jp


immutable ShapeFunction
    N::Function
    ∇N::Function
end

function ShapeFunction(local_coord::Vector, P::Function, ∇P::Function)
    coef = vcat([P(x...) for x=local_coord]...) |> inv
    N(point) = P(point...) * coef                            ######## Where does this "point" come from?
    ∇N(point) = ∇P(point...) * coef
    return ShapeFunction(N, ∇N)
end

function (shape::ShapeFunction)(point)
    return shape.N(point)
end

function (shape::ShapeFunction)(point, dim::Int64)           ####### What is this used for?
    base = shape(point)
    N = zeros(dim, dim*length(base))
    for i = 1:dim
        N[i,i:dim:end] = base
    end
    return N
end

function (shape::ShapeFunction)(::Type{Val{:grad}}, point)   ####### What is it for?
    return shape.∇N(point)
end

function (shape::ShapeFunction)(::Type{Val{:jacobian}}, point, geometry::Vector)
    ∇N = shape(Val{:grad}, point)
    return ∇N * hcat(geometry...)'
end

function (shape::ShapeFunction)(::Type{Val{:detj}}, point, geometry::Vector)
    J = shape(Val{:jacobian}, point, geometry)
    n, m = size(J)
    if n == m  # volume element
        return det(J)
    end
    Jᵀ = transpose(J)
    if size(Jᵀ, 2) == 1  # boundary of 2d problem, || ∂X/∂ξ ||
        return vecnorm(Jᵀ)
    else # manifold on 3d problem, || ∂X/∂ξ₁ × ∂X/∂ξ₂ ||
        return vecnorm(cross(Jᵀ[:,1], Jᵀ[:,2]))
    end
end

function (shape::ShapeFunction)(::Type{Val{:grad}}, point, geometry::Vector)
    ∇N = shape(Val{:grad}, point)
    J = shape(Val{:jacobian}, point, geometry)
    return inv(J) * ∇N
end
